In [ ]:
! pip install dspy-ai
! pip install transformers
! pip install accelerate
# !pip install "openai<1.0.0"
# # !pip install azure-keyvault-secrets azure-identity

In [1]:
import os
import json
import random

import dspy
import openai
import pandas as pd


from dsp.utils.utils import deduplicate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFinetune

kwargs={"max_tokens":1000}
lm = dspy.HFModel(model='mistralai/Mistral-7B-Instruct-v0.1')
dspy.settings.configure(lm=lm)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Word Scrambling

## Dataset

In [2]:
from datasets import load_dataset
from collections import defaultdict

hf_dataset = load_dataset("kurtn718/scrambled_words_multiple_choice")

# Function to convert a Huggingface dataset row to DSPy format
def convert_to_dspy_format(item):
    return dspy.Example(
        answer=item["correct_answer"],
        word=item["word"],
        scrambled=item["scrambled"],
        question=item["question"]
    )

# Convert and combine all splits
dspy_dataset = defaultdict(list)
for split in ['train', 'validation', 'test']:
    for item in hf_dataset[split]:
        dspy_example = convert_to_dspy_format(item)
        dspy_dataset[split].append(dspy_example)

train_set = dspy_dataset["train"]
validation_set = dspy_dataset["validation"]
test_set = dspy_dataset["test"]

train_set = [x.with_inputs('question') for x in train_set]
validation_set = [x.with_inputs('question') for x in validation_set]
test_set = [x.with_inputs('question') for x in test_set]

# Check the first element of the training set
example = train_set[0]
print(example)

Example({'answer': 'A', 'word': 'prophet', 'scrambled': 'prohept', 'question': "Rearrange the letters in 'prohept' to form the correct word.\nA: prophet\nB: jessica\nC: child\nD: respected"}) (input_keys={'question'})


In [3]:
class BasicQA(dspy.Signature):
    """Answer multiple choice questions from the given options A,B,C,D"""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="1 letter from A,B,C,D")

# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=example.question)

# Print the input and the prediction.
print(f"Question: {example.question}")
print(f"Predicted Answer: {pred.answer}")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Rearrange the letters in 'prohept' to form the correct word.
A: prophet
B: jessica
C: child
D: respected
Predicted Answer: Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Rearrange the letters in 'prohept' to form the correct word. A: prophet B: jessica C: child D: respected
Answer: B


In [112]:
lm.inspect_history(n=1)





Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Rearrange the letters in 'prohept' to form the correct word. A: prophet B: jessica C: child D: respected
Answer:Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Rearrange the letters in 'prohept' to form the correct word. A: prophet B: jessica C: child D: respected
Answer: B





In [4]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(BasicQA)
    def postprocess(self, prediction):
        # print("_______")
        # print(prediction)
        # print("_______")
        return prediction.split("Answer: ")[-1][0]
    
    def forward(self, question):
        prediction = self.generate_answer(question=question)
        prediction.answer = self.postprocess(prediction.answer)
        return prediction

In [5]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = CoT()

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=example.question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {example.question}")
# print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Rearrange the letters in 'prohept' to form the correct word.
A: prophet
B: jessica
C: child
D: respected
Predicted Answer: B


In [125]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_comprehension = Evaluate(devset=test_set, num_threads=32, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

evaluate_on_comprehension(generate_answer_with_chain_of_thought, metric=metric)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Rearrange the letters in 'lmiited' to form the correct word. A: physiology B: space C: wherever D: limited
Answer: D


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 0/2000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting

Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Decode the word 'ceczh'. A: promotions B: supply C: tissues D: czech
Answer: D: czech


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Answer multiple choice questions from the given options A,B,C,D

---

Follow the following format.

Question: ${question}
Answer: 1 letter from A,B,C,D

---

Question: Can you unscramble 'vatil'? A: nebraska B: terrace C: testing D: vital
Answer: D: vital

Question: What is the capital of France? A: Madrid B: Berlin C: Paris D: Rome
Answer: C: Paris

Question: Which planet is closest to the sun? A: Jupiter B: Venus C: Mercury D: Mars
Answer: C: Mercury

Question: What is the largest mammal on earth? A: elephant B: whale C: bear D: giraffe
Answer: B: whale

Question: What is the smallest country in the world? A: vatican city B: monaco C: san marino D: liechtenstein
Answer: A: vatican city

Question



Average Metric: 799 / 2000  (40.0): 100%|██████████| 2000/2000 [28:58<00:00,  1.15it/s]

Average Metric: 799 / 2000  (40.0%)


,example_answer,word,scrambled,question,pred_answer,answer_exact_match
0,B,knee,kene,Can you unscramble 'kene'? A: raymond B: knee C: take D: consumption,B,✔️ [True]
1,D,camcorders,cmorarecds,Decode the word 'cmorarecds'. A: charitable B: thats C: univ D: camcorders,D,✔️ [True]
2,C,heat,haet,Decode the word 'haet'. A: babes B: competent C: heat D: dogs,D,❌ [False]
3,C,brochure,bruchore,Can you unscramble 'bruchore'? A: acrylic B: monroe C: brochure D: products,C,✔️ [True]
4,B,grass,gasrs,Rearrange the letters in 'gasrs' to form the correct word. A: valuable B: grass C: fresno D: dept,D,❌ [False]


39.95

In [6]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_answer)

uncompiled_cot = CoT()

# Compile!
compiled_cot = teleprompter.compile(uncompiled_cot, trainset=train_set[:100])

  9%|▉         | 9/100 [00:48<08:11,  5.40s/it]

Bootstrapped 4 full traces after 10 examples in round 0.


In [7]:
for name, parameter in compiled_cot.named_predictors():
    print(name)
    for p in parameter.demos:
        print(p)
    print()

generate_answer
Example({'augmented': True, 'question': "Decode the word 'beer'.\nA: eleven\nB: exercise\nC: birth\nD: beer", 'answer': 'D'}) (input_keys=None)
Example({'augmented': True, 'question': "Decode the word 'porgtahpohy'.\nA: orleans\nB: negotiate\nC: kelly\nD: photography", 'answer': 'D'}) (input_keys=None)
Example({'augmented': True, 'question': "Can you unscramble 'fhtuerr'?\nA: mere\nB: totals\nC: north\nD: further", 'answer': 'D'}) (input_keys=None)
Example({'augmented': True, 'question': "What word does 'srting' represent when unscrambled?\nA: contracting\nB: fishing\nC: sheraton\nD: string", 'answer': 'D'}) (input_keys=None)
Example({'answer': 'C', 'word': 'neat', 'scrambled': 'naet', 'question': "Find the right word for 'naet'.\nA: galleries\nB: disabled\nC: neat\nD: changed"}) (input_keys={'question'})
Example({'answer': 'B', 'word': 'reforms', 'scrambled': 'remfros', 'question': "Rearrange the letters in 'remfros' to form the correct word.\nA: decorative\nB: reforms

In [8]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_comprehension = Evaluate(devset=test_set, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

evaluate_on_comprehension(compiled_cot, metric=metric)

Average Metric: 508 / 2000  (25.4): 100%|██████████| 2000/2000 [2:58:41<00:00,  5.36s/it]

Average Metric: 508 / 2000  (25.4%)



/opt/conda/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:126: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,example_answer,word,scrambled,question,pred_answer,answer_exact_match
0,B,knee,kene,Can you unscramble 'kene'? A: raymond B: knee C: take D: consumption,D,❌ [False]
1,D,camcorders,cmorarecds,Decode the word 'cmorarecds'. A: charitable B: thats C: univ D: camcorders,D,✔️ [True]
2,C,heat,haet,Decode the word 'haet'. A: babes B: competent C: heat D: dogs,D,❌ [False]
3,C,brochure,bruchore,Can you unscramble 'bruchore'? A: acrylic B: monroe C: brochure D: products,D,❌ [False]
4,B,grass,gasrs,Rearrange the letters in 'gasrs' to form the correct word. A: valuable B: grass C: fresno D: dept,D,❌ [False]


25.4

## Finetune T5

In [12]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_scramble = Evaluate(devset=test_set, num_threads=2, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

In [13]:
# tp = BootstrapFewShotWithRandomSearch(metric=validate_answer, max_bootstrapped_demos=2, num_threads=2)
tp = BootstrapFewShot(metric=validate_answer, max_bootstrapped_demos=4)

cot_bs = tp.compile(CoT(), trainset=train_set[:50], valset=validation_set[50:200])

  8%|▊         | 4/50 [00:00<00:00, 1407.96it/s]

Bootstrapped 4 full traces after 5 examples in round 0.


In [14]:
cot_bs.save(f'cot_gpt35.json')

In [15]:
prog = CoT()
prog.load(f'cot_gpt35.json')
ensemble = [prog]


In [16]:
always_true = lambda g, p, trace=None: True

In [17]:
!pip install evaluate
!pip install rouge_score
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [18]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [27]:
config = dict(target='google/flan-t5-small', epochs=2, bf16=True, bsize=6, accumsteps=2, lr=5e-5)

tp = BootstrapFinetune(metric=None)
t5_program = tp.compile(CoT(), teacher=ensemble, trainset=test_set[:100], **config)

# Deactivate chain of thought prompting. Let's use T5 to directly predict outputs. (Faster and similar quality.)
for p in t5_program.predictors(): p.activated = False

100%|██████████| 100/100 [00:00<00:00, 3776.03it/s]


Bootstrapped 100 full traces after 100 examples in round 0.
all 100
local_cache/compiler/all.cfeec7cbd49cfd3e.jsonl


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset statistics: {'max_source_length': 44, 'max_target_length': 2}
Keys of tokenized dataset: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels']
Finetuning dataset: DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 90
    })
    test: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})
{'eval_loss': 0.6288124322891235, 'eval_rouge1': 30.0, 'eval_rouge2': 0.0, 'eval_rougeL': 30.0, 'eval_rougeLsum': 30.0, 'eval_gen_len': 2.0, 'eval_runtime': 0.1223, 'eval_samples_per_second': 81.734, 'eval_steps_per_second': 16.347, 'epoch': 0.93}
{'eval_loss': 0.6246126890182495, 'eval_rouge1': 30.0, 'eval_rouge2': 0.0, 'eval_rougeL': 30.0, 'eval_rougeLsum': 30.0, 'eval_gen_len': 2.0, 'eval_runtime': 0.1443, 'eval_samples_per_second': 69.281, 'eval_steps_per_second': 13.856, 'epoch': 1.87}
{'train_runtime': 3.4201, 'train_samp

In [28]:
t5_program_load = CoT()
ckpt_path = '../finetuning_ckpts/S32P3WSDB9KH7.all/checkpoint-14'
# ckpt_path = "colbert-ir/dspy-Oct11-T5-Large-MH-3k-v1"
LM = dspy.HFModel(checkpoint=ckpt_path, model='google/flan-t5-small')

for p in t5_program_load.predictors():
    p.lm = LM
    p.activated = False

In [29]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_scramble = Evaluate(devset=test_set, num_threads=2, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

In [31]:
score = evaluate_on_scramble(t5_program_load, metric=metric)

Average Metric: 916 / 2000  (45.8): 100%|██████████| 2000/2000 [00:22<00:00, 89.12it/s]


Average Metric: 916 / 2000  (45.8%)


,example_answer,word,scrambled,question,pred_answer,answer_exact_match
0,B,knee,kene,Can you unscramble 'kene'? A: raymond B: knee C: take D: consumption,C,❌ [False]
1,D,camcorders,cmorarecds,Decode the word 'cmorarecds'. A: charitable B: thats C: univ D: camcorders,A,❌ [False]
2,C,heat,haet,Decode the word 'haet'. A: babes B: competent C: heat D: dogs,B,❌ [False]
3,C,brochure,bruchore,Can you unscramble 'bruchore'? A: acrylic B: monroe C: brochure D: products,D,❌ [False]
4,B,grass,gasrs,Rearrange the letters in 'gasrs' to form the correct word. A: valuable B: grass C: fresno D: dept,A,❌ [False]


In [30]:
# Call the predictor on a particular input.
pred = t5_program(question=example.question)

# Print the input and the prediction.
print(f"Question: {example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: Rearrange the letters in 'prohept' to form the correct word.
A: prophet
B: jessica
C: child
D: respected
Predicted Answer: D


In [ ]:
t5_program.predictors()[0].lm.inspect_history(n=3)

In [9]:
! ls
! rm -rf RACE
! apt install unzip
! unzip -q RACE.zip
! rm -rf __MACOSX
! ls

'Model_Wranglers_DSPy (1).ipynb'   RACE.zip   local_cache
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 111 not upgraded.
'Model_Wranglers_DSPy (1).ipynb'   RACE   RACE.zip   __MACOSX   local_cache


## Dataset

In [13]:
! ls RACE/dev

high  middle


In [31]:
import os
import json
from collections import defaultdict
import dspy

# Function to convert a RACE dataset row to DSPy format
def convert_race_to_dspy_format(item):
    context = item["article"]
    options = item["options"]
    questions = item["questions"]
    answers = item["answers"]

    # Assuming options, questions, and answers have the same length
    examples = []
    answer_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    for i in range(len(options)):
        example = dspy.Example(
            context=context,
            question=questions[i],
            options=str(options[i]),
            answer_option=answers[i],
            answer = options[i][answer_to_index[answers[i]]]
        )
        examples.append(example)
    
    return examples

# Convert and combine all splits for middle and high school
dspy_dataset = defaultdict(lambda: defaultdict(list))

for level in ['middle', 'high']:
    for split in ['dev', 'test', 'train']:
        folder_path = os.path.join("RACE", split, level)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                dspy_examples = convert_race_to_dspy_format(data)
                dspy_dataset[level][split].extend(dspy_examples)

# Add inputs to the datasets
for level in ['middle', 'high']:
    for split in ['dev', 'test', 'train']:
        dspy_dataset[level][split] = [x.with_inputs('context', 'question', 'options') for x in dspy_dataset[level][split]]

# Check the first element of the datasets
example_middle_school = dspy_dataset["middle"]["dev"][0]
example_high_school = dspy_dataset["high"]["dev"][0]

print("middle School Example:")
print(example_middle_school)
print("\nHigh School Example:")
print(example_high_school)


middle School Example:
Example({'context': "Do you know HFMD? It's short for Hand-Foot-Mouth Disease. This year, in China, thousands of children were suffering from it. What's HFMD? How to prevent it? Now, read the passage please.\nHFMD usually affects babies who are 1~4 years old, but adults can also be infected. Both EV71 and Cox A16 can cause HFMD, which usually starts with a slight fever followed by blisters   and ulcers   in the mouth and rashes   on the hands and feet.\nIt can be spread through people with the mucus   or feces   of an _ person. It usually appears during the summer and autumn months. HFMD isn't Bird Flu, SARS or Mad Cow Disease, but it's not a new one, either. It first appeared in New Zealand in 1957. About forty years later, it appears in Asia. It's reported that it breaks out every 2 or 3 years.\nHFMD is very terrible and there is no vaccine   now, but we can do something helpful to prevent it. 'Children with HFMD should seek medical treatment as early as possib

## DSPy Module

In [71]:
"""Given the context, fill in the  _ by choosing from given options. Give one letter answer. A,B,C,D"""
class BasicQA(dspy.Signature):
    """Given the context, answer the question by choosing from given options."""

    context = dspy.InputField()
    question = dspy.InputField()
    options = dspy.InputField(desc="4 options to select from")
    answer = dspy.OutputField()
    
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(BasicQA)
    def postprocess(self, prediction):
        # print("_______")
        # print(prediction)
        # print("_______")
        return prediction.split("Answer: ")[-1].split('\n')[0]

    def forward(self, context, question, options):
        prediction = self.generate_answer(context=context, question=question, options=options, kwargs={"max_tokens":2000})
        prediction.answer = self.postprocess(prediction.answer)
        return prediction

In [72]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = CoT()

example = example_high_school
for example in dspy_dataset["middle"]["test"][9:10]:
    # Call the predictor on the same input.
    pred = generate_answer_with_chain_of_thought(
        context=example.context,
        question=example.question,
        options=example.options
    )
    
    # Print the input, the chain of thought, and the prediction.
    print(f"Question: {example.question}")
    print(f"Correct Answer: {example.answer}")
    # print(f"Thought: {pred.rationale}") #.split('.', 1)[1].strip()}")
    print(f"Predicted Answer: {pred.answer}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Which of these is not right?
Correct Answer: John exercises on Sundays.
Predicted Answer: ${answer}


In [54]:
lm.inspect_history(n=1)





Given the context, answer the question by choosing from given options.

---

Follow the following format.

Context: ${context}

Question: ${question}

Options: 4 options to select from

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
John gets up early from Monday to Saturday, because he must go to school before 7:30 on weekdays  and go to Drawing Club at 8:00 on Saturday mornings. He usually goes to the bookshop on Saturday afternoon, and after dinner he watches TV until  midnight .
He doesn't get up early on Sundays. John's parents both work on Sundays. John always watches TV after he gets up. Then he usually goes to KFC to have a hamburger and some juice for lunch. After that, he goes back home and starts to play computer games until his parents come back. He does his homework after dinner. He usually has lots of weekend homework, so he must spend three hours on it. He usually goes to bed at about 11:00 p.m. on Sund

In [74]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_comprehension = Evaluate(devset=dspy_dataset["middle"]["test"], num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

evaluate_on_comprehension(generate_answer_with_chain_of_thought, metric=metric)



  0%|          | 0/1436 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 1 / 1  (100.0):   0%|          | 1/1436 [00:01<31:07,  1.30s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 2  (100.0):   0%|          | 2/1436 [00:02<31:52,  1.33s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 3  (66.7):   0%|          | 3/1436 [00:06<1:02:55,  2.63s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 4  (50.0):   0%|          | 4/1436 [00:10<1:13:31,  3.08s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 3 / 5  (60.0):   0%|          | 5/1436 [00:15<1:25:25,  3.58s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 3 / 6  (50.0):   0%|          | 6/1436 [00:17<1:19:59,  3.36s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end gener

Average Metric: 452 / 1436  (31.5%)


,context,question,options,answer_option,example_answer,rationale,pred_answer,answer_exact_match
0,My name is Nancy. I'm twelve years old. I have a good friend. Her name is Wendy. She is thirteen. She is from Australia. We...,Where is Wendy from?,"['China.', 'England.', 'America.', 'Australia.']",D,Australia.,"Given the context, answer the question by choosing from given options. --- Follow the following format. Context: ${context} Question: ${question} Options: 4 options to select...",Australia.,✔️ [True]
1,My name is Nancy. I'm twelve years old. I have a good friend. Her name is Wendy. She is thirteen. She is from Australia. We...,What colours does Nancy like?,"['Red and blue.', 'Red and yellow.', 'Green and yellow.', 'Green and blue.']",D,Green and blue.,"Given the context, answer the question by choosing from given options. --- Follow the following format. Context: ${context} Question: ${question} Options: 4 options to select...",Green and blue.,✔️ [True]
2,My name is Nancy. I'm twelve years old. I have a good friend. Her name is Wendy. She is thirteen. She is from Australia. We...,What's Wendy's favourite sport?,"['Running.', 'Basketball.', 'Football.', 'Table tennis.']",A,Running.,"Given the context, answer the question by choosing from given options. --- Follow the following format. Context: ${context} Question: ${question} Options: 4 options to select...",We don't have enough information to answer the question.,❌ [False]
3,My name is Nancy. I'm twelve years old. I have a good friend. Her name is Wendy. She is thirteen. She is from Australia. We...,Which is TRUE ?,"['Nancy and Wendy are 12 years old.', 'Wendy is a student and she is English.', 'Everyone in Class Four likes Wendy.', 'Nancy has a cat...",C,Everyone in Class Four likes Wendy.,"Given the context, answer the question by choosing from given options. --- Follow the following format. Context: ${context} Question: ${question} Options: 4 options to select...",Nancy has a dog and Wendy has a cat.,❌ [False]
4,"June 5 is World Environment Day.This makes us pay more attention to our environment and the need to protect it. When Wang Baoxuan,a Beijing high...",What do Wang Baoxuan and his schoolmates do with the waste exercise books?,"['Throw them away', 'Collect and sell them', 'Cut them into pieces', 'Give them to the students in Inner Mongolia']",B,Collect and sell them,"Given the context, answer the question by choosing from given options. --- Follow the following format. Context: ${context} Question: ${question} Options: 4 options to select...",Collect and sell them.,✔️ [True]


31.48

In [75]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_comprehension = Evaluate(devset=dspy_dataset["high"]["test"], num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

evaluate_on_comprehension(generate_answer_with_chain_of_thought, metric=metric)



  0%|          | 0/3498 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 1 / 1  (100.0):   0%|          | 1/3498 [00:03<3:14:09,  3.33s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 2  (100.0):   0%|          | 2/3498 [00:06<3:01:01,  3.11s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 3  (66.7):   0%|          | 3/3498 [00:11<3:48:15,  3.92s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 4  (50.0):   0%|          | 4/3498 [00:16<4:20:44,  4.48s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 5  (40.0):   0%|          | 5/3498 [00:20<4:14:34,  4.37s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Average Metric: 2 / 6  (33.3):   0%|          | 6/3498 [00:25<4:31:45,  4.67s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end g

KeyboardInterrupt: 

In [ ]:
## Compilation

In [ ]:
# 